# Advanced pyALF part 1: Running ALF

In [1]:
from pprint import pprint  # Pretty print
from py_alf import ALF_source, Simulation  # Interface with ALF

## ALF_source
Create instance of `ALF_source`, choosing to checkout the git branch `'211-add-nematic-dirac-hamiltonian'`, since the Nematic Dirac Hamiltonian is not on master.

Reminder: Directory containing the ALF code is taken from environment variable `$ALF_DIR`, if present.

In [2]:
alf_src = ALF_source(
    branch='211-add-nematic-dirac-hamiltonian',
)

Checking out branch 211-add-nematic-dirac-hamiltonian
Your branch is up to date with 'origin/211-add-nematic-dirac-hamiltonian'.


Already on '211-add-nematic-dirac-hamiltonian'


Check available Hamiltonians:

In [3]:
alf_src.get_ham_names()

['Kondo',
 'Hubbard',
 'Hubbard_Plain_Vanilla',
 'tV',
 'LRC',
 'Z2_Matter',
 'Nematic_Dirac']

Print valid parameters in their defaults for Nematic Dirac Hamiltonian, including QMC parameters:

In [4]:
pprint(alf_src.get_default_params('Nematic_Dirac'))

OrderedDict([('VAR_Nematic_Dirac',
              {'Global_J': {'comment': 'J for proposing global updates',
                            'defined_in_base': False,
                            'value': 1.0},
               'Global_h': {'comment': 'h for proposing global updates',
                            'defined_in_base': False,
                            'value': 3.0},
               'Global_type': {'comment': 'Type of global update. Possible '
                                          "values: 'Wolff', 'Geo', 'switch', "
                                          "'flip'",
                               'defined_in_base': False,
                               'value': ''},
               'Ham_J': {'comment': 'Ferromagnetic Ising interaction',
                         'defined_in_base': False,
                         'value': 1.0},
               'Ham_chem': {'comment': 'Chemical potential',
                            'defined_in_base': False,
                            'value': 0

## Run a series of simulations
Create a list of `Simulation` instances with varying transverse Ising field $h$.

MPI parallelization is enabled, with number of processes set to 4. This results in 4 quasi-independent runs with the same parameters and therefore more precise outcomes.

In [3]:
L = 4
simsL4 = [
    Simulation(
        alf_src,
        'Nematic_Dirac',
        {
            # Model specific parameters
            'Model_vers': 1,
            'L1': L,
            'L2': L,
            'beta': L*4.,
            'Ham_xi': 0.25,
            'Ham_h': h,
            # QMC parameters
            'Ltau': 1,
            'CPU_MAX': .1,
            'NSweep': 20,
        },
        machine='intel',
        mpi=True,
        n_mpi=4,
        #mpiexec='orterun',
    )
    for h in [2.5, 3.0, 3.5, 4.0]]

In [4]:
simsL4

In [5]:
# Simple example for Pythons list comprehension (and f-strings)
[f'h={h}' for h in [2.5, 3.0, 3.5, 4.0]]

['h=2.5', 'h=3.0', 'h=3.5', 'h=4.0']

`Simulation` generates the directory name the simulation is run in out of the non-default parameters, prepended by "ALF_data". For the first simulation, this is:

In [6]:
for sim in simsL4:
    print(sim.get_directories())

['/scratch/Nematic_example/ALF_data/Nematic_Dirac_Model_vers=1_L1=4_L2=4_beta=16.0_xi=0.25_h=2.5']
['/scratch/Nematic_example/ALF_data/Nematic_Dirac_Model_vers=1_L1=4_L2=4_beta=16.0_xi=0.25_h=3.0']
['/scratch/Nematic_example/ALF_data/Nematic_Dirac_Model_vers=1_L1=4_L2=4_beta=16.0_xi=0.25_h=3.5']
['/scratch/Nematic_example/ALF_data/Nematic_Dirac_Model_vers=1_L1=4_L2=4_beta=16.0_xi=0.25_h=4.0']


This behaviour can be overwritten through the optional arguments `sim_dir` and `sim_root`.

Compile ALF.

In [10]:
simsL4[0].compile()

Compiling ALF... 
Cleaning up Prog/
Cleaning up Libraries/
Cleaning up Analysis/
Compiling Libraries


ar: creating modules_90.a
ar: creating libqrref.a
ana_mod.F90(1213): warning #5268: Extension to standard: The text exceeds right hand column allowed on the line.
            Write(error_unit,*) 'Evaluating the mutual information between A and B requires the entanglement entropies of A, B and the union of A and B, i.e. Nobs=4 (3 + 1 for the phase)'
-------------------------------------------------------------------------------------------------------------------------------------^


Compiling Analysis


ifort: command line remark #10412: option '-mkl' is deprecated and will be removed in a future release. Please use the replacement option '-qmkl'
ifort: command line remark #10412: option '-mkl' is deprecated and will be removed in a future release. Please use the replacement option '-qmkl'
ifort: command line remark #10412: option '-mkl' is deprecated and will be removed in a future release. Please use the replacement option '-qmkl'
Max_SAC.F90(164): warning #5268: Extension to standard: The text exceeds right hand column allowed on the line.
       If (  nbin_qmc < 2*Ntau .and. N_cov == 1  )   Write(50,*) 'You do not seem to have enough bins for a reliable estimate of the covariance '
-------------------------------------------------------------------------------------------------------------------------------------^
ifort: command line remark #10412: option '-mkl' is deprecated and will be removed in a future release. Please use the replacement option '-qmkl'
ifort: command line rem

Compiling Program
Parsing Hamiltonian parameters
filename: Hamiltonians/Hamiltonian_Kondo_smod.F90
filename: Hamiltonians/Hamiltonian_Hubbard_smod.F90
filename: Hamiltonians/Hamiltonian_Hubbard_Plain_Vanilla_smod.F90
filename: Hamiltonians/Hamiltonian_tV_smod.F90
filename: Hamiltonians/Hamiltonian_LRC_smod.F90
filename: Hamiltonians/Hamiltonian_Z2_Matter_smod.F90
filename: Hamiltonians/Hamiltonian_Nematic_Dirac_smod.F90
Compiling program modules


Operator_mod.F90(143): warning #5268: Extension to standard: The text exceeds right hand column allowed on the line.
!> * On exit:  phase of  \f$ W(C)  =   \left[ \left( \prod_{n,\tau,f}  \exp \left[ g_f(n) \alpha_f(n) \phi(\sigma(n,\tau)) \right] \right) \det(M_f(C))\right]^{N_{SUN}} \prod_{n,\tau }\gamma(\sigma(n,\tau)) \f$
------------------------------------------------------------------------------------------------------------------------------------^
Predefined_Obs_mod.F90(347): warning #5268: Extension to standard: The text exceeds right hand column allowed on the line.
!>  Returns:  \frac{2N}{N^2-1}\sum_{a=1}{N^2 - 1 }  <c^{dag}_i(tau) T^a c_i (tau)  c^{dag}_j T^a  c_j>  where  T^a are th generators of SU(N)
------------------------------------------------------------------------------------------------------------------------------------^
./Cotunneling_dimer_obs.F90(226): warning #5268: Extension to standard: The text exceeds right hand column allowed on the line.
           

Link program


ifort: command line remark #10412: option '-mkl' is deprecated and will be removed in a future release. Please use the replacement option '-qmkl'


Done.


Run the list of simulations in sequence.

In [11]:
for sim in simsL4:
    sim.run()

Prepare directory "/scratch/Nematic_example/ALF_data/Nematic_Dirac_Model_vers=1_L1=4_L2=4_beta=16.0_xi=0.25_h=2.5" for Monte Carlo run.
Create new directory.
Run /home/jschwab/Programs/ALF/Prog/ALF.out
 ALF Copyright (C) 2016 - 2022 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
IPL WARN> ipl_option_domain: wrong layout value cache3
Prepare directory "/scratch/Nematic_example/ALF_data/Nematic_Dirac_Model_vers=1_L1=4_L2=4_beta=16.0_xi=0.25_h=3.0" for Monte Carlo run.
Create new directory.
Run /home/jschwab/Programs/ALF/Prog/ALF.out
 ALF Copyright (C) 2016 - 2022 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
IPL WARN> ipl_option_domain: wrong la

Check info files produced by ALF. It is strongly advised to take a look at these after finished runs, in particular "Precision Green" and "Precision Phase". These should be small, of order $10^{-8}$ or smaller. If they're bigger, one should decrease the stabilization invervall `Nwrap` (see parameter list `'VAR_QMC'` above). In our case, they're of order $10^{-14}$ and one might consider increasing `Nwrap` to speed up the simulation.

In [7]:
for sim in simsL4:
    sim.print_info_file()

===== /scratch/Nematic_example/ALF_data/Nematic_Dirac_Model_vers=1_L1=4_L2=4_beta=16.0_xi=0.25_h=2.5/info =====
 Model is            : Nematic_Dirac           1
 Global_type         :                                                                 
 L1                  :            4
 L2                  :            4
 N_SUN               :            2
 ham_t               :    1.00000000000000     
 dtau                :   0.100000000000000     
 beta                :    16.0000000000000     
 Ham_h               :    2.50000000000000     
 Ham_J               :    1.00000000000000     
 Ham_xi              :   0.250000000000000     
 Ham_xi2             :   0.000000000000000E+000
 Ham_chem            :   0.000000000000000E+000
 Global_J            :    1.00000000000000     
 Global_h            :    3.00000000000000     
 Phi_1               :   0.000000000000000E+000
 Phi_2               :   0.000000000000000E+000
 init_type           : random                                      

## Parallel Tempering
The simulation object below shows an example of Parallel Tempering, which runs simulations with different parameters in parallel and exchanges configurations between them. The method has been developed to address ergodicity issues, but in this case it's only used to execute a parameter sweep "in one go". For more techincal details on Parallel Tempering, refer to the ALF documentation.

Parallel Tempering is enabled by making `sim_dict` a list of dictionaries, instead of a single dictionary.

In [8]:
L = 6
simL6 = Simulation(
    alf_src,
    'Nematic_Dirac',
    [{
        # Model specific parameters
        'Model_vers': 1,
        'L1': L,
        'L2': L,
        'beta': L*4.,
        'Ham_xi': 0.25,
        'Ham_h': h,
        # QMC parameters
        'Ltau': 1,
        'CPU_MAX': 1.,
        'NSweep': 20,
        'mpi_per_parameter_set': 1,
        'Tempering_calc_det': False,
    } for h in [2.5, 3.0, 3.5, 4.0]],
    machine='intel',
    mpi=True,
    n_mpi=4,
    #mpiexec='orterun',
)

In [10]:
simL6.get_directories()

['/scratch/Nematic_example/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=6_L2=6_beta=24.0_xi=0.25_h=2.5/Temp_0',
 '/scratch/Nematic_example/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=6_L2=6_beta=24.0_xi=0.25_h=2.5/Temp_1',
 '/scratch/Nematic_example/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=6_L2=6_beta=24.0_xi=0.25_h=2.5/Temp_2',
 '/scratch/Nematic_example/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=6_L2=6_beta=24.0_xi=0.25_h=2.5/Temp_3']

In [ ]:
simL6.run()

In [9]:
simL6.print_info_file()

===== /scratch/Nematic_example/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=6_L2=6_beta=24.0_xi=0.25_h=2.5/Temp_0/info =====
 Model is            : Nematic_Dirac           1
 Global_type         :                                                                 
 L1                  :            6
 L2                  :            6
 N_SUN               :            2
 ham_t               :    1.00000000000000     
 dtau                :   0.100000000000000     
 beta                :    24.0000000000000     
 Ham_h               :    2.50000000000000     
 Ham_J               :    1.00000000000000     
 Ham_xi              :   0.250000000000000     
 Ham_xi2             :   0.000000000000000E+000
 Ham_chem            :   0.000000000000000E+000
 Global_J            :    1.00000000000000     
 Global_h            :    3.00000000000000     
 Phi_1               :   0.000000000000000E+000
 Phi_2               :   0.000000000000000E+000
 init_type           : random                        

In [16]:
L = 8
simL8 = Simulation(
    alf_src,
    'Nematic_Dirac',
    [{
        # Model specific parameters
        'Model_vers': 1,
        'L1': L,
        'L2': L,
        'beta': L*4.,
        'Ham_xi': 0.25,
        'Ham_h': h,
        # QMC parameters
        'Ltau': 1,
        'CPU_MAX': 7.,
        'NSweep': 20,
        'Nbin': 1,
        'mpi_per_parameter_set': 1,
        'Tempering_calc_det': False,
    } for h in [2.5, 3.0, 3.5, 4.0]],
    machine='intel',
    mpi=True,
    n_mpi=4,
    #mpiexec='orterun',
)

In [ ]:
simL8.run()

In [17]:
simL8.print_info_file()

===== /scratch/Nematic_example/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=8_L2=8_beta=32.0_xi=0.25_h=2.5/Temp_0/info =====
 Model is            : Nematic_Dirac           1
 Global_type         :                                                                 
 L1                  :            8
 L2                  :            8
 N_SUN               :            2
 ham_t               :    1.00000000000000     
 dtau                :   0.100000000000000     
 beta                :    32.0000000000000     
 Ham_h               :    2.50000000000000     
 Ham_J               :    1.00000000000000     
 Ham_xi              :   0.250000000000000     
 Ham_xi2             :   0.000000000000000E+000
 Ham_chem            :   0.000000000000000E+000
 Global_J            :    1.00000000000000     
 Global_h            :    3.00000000000000     
 Phi_1               :   0.000000000000000E+000
 Phi_2               :   0.000000000000000E+000
 init_type           : random                        

In [20]:
L = 10
simL10 = Simulation(
        alf_src,
        'Nematic_Dirac',
        [{
            # Model specific parameters
            'Model_vers': 1,
            'L1': L,
            'L2': L,
            'beta': L*4.,
            'Ham_xi': 0.25,
            'Ham_h': h,
            # QMC parameters
            'Ltau': 1,
            'CPU_MAX': 18,
            'NSweep': 20,
            'mpi_per_parameter_set': 1,
        } for h in [2.5, 3.0, 3.5, 4.0]],
        machine='intel',
        mpi=True,
        n_mpi=4,
        #mpiexec='orterun',
    )

In [ ]:
simL10.run()

In [21]:
simL10.print_info_file()

===== /scratch/Nematic_example/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=10_L2=10_beta=40.0_xi=0.25_h=2.5/Temp_0/info =====
 Model is            : Nematic_Dirac           1
 Global_type         :                                                                 
 L1                  :           10
 L2                  :           10
 N_SUN               :            2
 ham_t               :    1.00000000000000     
 dtau                :   0.100000000000000     
 beta                :    40.0000000000000     
 Ham_h               :    2.50000000000000     
 Ham_J               :    1.00000000000000     
 Ham_xi              :   0.250000000000000     
 Ham_xi2             :   0.000000000000000E+000
 Ham_chem            :   0.000000000000000E+000
 Global_J            :    1.00000000000000     
 Global_h            :    3.00000000000000     
 Phi_1               :   0.000000000000000E+000
 Phi_2               :   0.000000000000000E+000
 init_type           : random                      

In [22]:
## Only preparing runs

In many cases, it might not be feasible to execute ALF directly through pyALF, for example when using an HPC scheduler, but one might still like to use pyALF for preparing the simulation directories. In this case the two options `copy_bin` and `only_prep` of {func}`py_alf.Simulation.run` come in handy. This will also serve as a demonstration of the keyword arguments `sim_root` and `sim_dir`.

In [15]:
L = 20
simsL20 = [
    Simulation(
        alf_src,
        'Nematic_Dirac',
        {
            # Model specific parameters
            'Model_vers': 1,
            'L1': L,
            'L2': L,
            'beta': L*4.,
            'Ham_xi': 0.25,
            'Ham_h': h,
            # QMC parameters
            'Ltau': 1,
            'CPU_MAX': 48,
            'NSweep': 20,
        },
        machine='intel',
        mpi=True,
        n_mpi=4,
        sim_root="~/NematicDiracL20",
        sim_dir=f"h{h:2.2f}",
    )
    for h in [3.1, 3.2, 3.3, 3.4, 3.5]]

In [ ]:
simsL20.compile()

In [16]:
for sim in simsL20:
    sim.run(copy_bin=True, only_prep=True)

Prepare directory "/home/jschwab/NematicDiracL20/h3.10" for Monte Carlo run.
Create new directory.
Prepare directory "/home/jschwab/NematicDiracL20/h3.20" for Monte Carlo run.
Create new directory.
Prepare directory "/home/jschwab/NematicDiracL20/h3.30" for Monte Carlo run.
Create new directory.
Prepare directory "/home/jschwab/NematicDiracL20/h3.40" for Monte Carlo run.
Create new directory.
Prepare directory "/home/jschwab/NematicDiracL20/h3.50" for Monte Carlo run.
Create new directory.


In [14]:
ls NematicDiracL20/*

NematicDiracL20/h3.10:
ALF.out*  parameters  seeds

NematicDiracL20/h3.20:
ALF.out*  parameters  seeds

NematicDiracL20/h3.30:
ALF.out*  parameters  seeds

NematicDiracL20/h3.40:
ALF.out*  parameters  seeds

NematicDiracL20/h3.50:
ALF.out*  parameters  seeds
